In [1]:
import os
import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from google.colab import files
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow

In [2]:
! pip install -q kaggle
files.upload() #upload kaggle.json file
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
def downloadKaggleData(link):
  name = (link.split(' ')[-1]).split('/')[-1]
  ! {link}
  ! mkdir {name}
  ! unzip {name}.zip -d {name}
downloadKaggleData("kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")

Dataset URL: https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
License(s): CC0-1.0
gtsrb-german-traffic-sign.zip: Skipping, found more recently modified local copy (use --force to force download)
mkdir: cannot create directory ‘gtsrb-german-traffic-sign’: File exists
Archive:  gtsrb-german-traffic-sign.zip
replace gtsrb-german-traffic-sign/Meta.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader, Sampler
from PIL import Image
import torchvision

class MyGTSRBset(Dataset):
  def __init__(self, lablecsv, transform = None):
    csvread = pd.read_csv(lablecsv)
    self.image_files = []
    self.imagelables = csvread['ClassId']
    self.transform = transform
    imagepaths = csvread['Path']
    for imagepath in imagepaths:
      imagepath = 'gtsrb-german-traffic-sign/' + imagepath
      image = Image.open(imagepath)
      image = transform(image)
      image = torch.flatten(image)
      self.image_files.append(image)

  def __len__(self):
    return len(self.image_files)

  def __getitem__(self, idx):
    imagepath = self.image_files[idx]

    return self.image_files[idx], self.imagelables[idx]

transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((32, 32)),
    torchvision.transforms.PILToTensor()

])
testdataset = MyGTSRBset('gtsrb-german-traffic-sign/Test.csv', transform = transform)
traindataset = MyGTSRBset('gtsrb-german-traffic-sign/Train.csv', transform = transform)

In [5]:
print(traindataset[0])

(tensor([46, 48, 49,  ..., 73, 61, 62], dtype=torch.uint8), 20)


In [6]:
batch_size = 512
train_loader = DataLoader(traindataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(testdataset,  batch_size=batch_size, shuffle=False)

for X, y in train_loader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([512, 3072])
Shape of y: torch.Size([512]) torch.int64


In [10]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device} device')

class MLP(nn.Module):
  def __init__(self) -> None:
    super(MLP, self).__init__()
    inputlayer = [32*32*3]
    hiddenlayer = [1024,512,256,128]
    outputlayer = 43
    hiddenlayer = inputlayer + hiddenlayer

    self.layers = nn.Sequential()
    for i in range(len(hiddenlayer)-1):
      self.layers.add_module(f'linear_{i+1}', nn.Linear(hiddenlayer[i], hiddenlayer[i+1]))
      self.layers.add_module(f'relu_{i+1}', nn.ReLU())
    self.layers.add_module(f'linear_last', nn.Linear(hiddenlayer[-1], outputlayer))

  def forward(self, x):
    x = x.float()
    return self.layers(x)

model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
print(model)

Using cpu device
MLP(
  (layers): Sequential(
    (linear_1): Linear(in_features=3072, out_features=1024, bias=True)
    (relu_1): ReLU()
    (linear_2): Linear(in_features=1024, out_features=512, bias=True)
    (relu_2): ReLU()
    (linear_3): Linear(in_features=512, out_features=256, bias=True)
    (relu_3): ReLU()
    (linear_4): Linear(in_features=256, out_features=128, bias=True)
    (relu_4): ReLU()
    (linear_last): Linear(in_features=128, out_features=43, bias=True)
  )
)


In [8]:
def train(dataloader, model, loss_fn, optimizer):
  model.train()

  num_batches = len(dataloader)
  num_items = len(dataloader.dataset)

  total_loss = 0
  total_correct = 0

  losses = []
  for batch_num, (X, y) in enumerate(dataloader):
    optimizer.zero_grad()
    X, y = X.to(device), y.to(device)

    output = model(X)

    loss = loss_fn(output, y)

    total_loss += loss
    total_correct += (torch.argmax(output,dim = 1)== y).sum().item()
    loss.backward()
    losses.append(loss.item())
    optimizer.step()

  train_loss = total_loss/num_batches
  accuracy = total_correct/num_items

  print(f"Average loss: {train_loss:7f}, accuracy: {accuracy:.2%}")



In [19]:
epochs = 10
for epoch in range(epochs):
    print(f"Training epoch: {epoch+1}")
    train(train_loader, model, criterion, optimizer)

Training epoch: 1
Average loss: 0.296044, accuracy: 91.62%
Training epoch: 2
Average loss: 0.249363, accuracy: 93.18%
Training epoch: 3
Average loss: 0.315009, accuracy: 90.98%
Training epoch: 4
Average loss: 0.224721, accuracy: 93.76%
Training epoch: 5


KeyboardInterrupt: 

In [20]:
def test(dataloader, model, loss_fn):
  model.eval()

  num_batches = len(dataloader)
  num_items = len(dataloader.dataset)
  total_loss = 0
  total_correct = 0

  losses = []

  with torch.no_grad():
    for batch_num, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)

      output = model(X)

      loss = loss_fn(output, y)

      total_loss += loss
      total_correct += (torch.argmax(output,dim = 1)== y).sum().item()
      losses.append(loss.item())

  test_loss = total_loss/num_batches
  accuracy = total_correct/num_items

  print(f"Average loss: {test_loss:7f}, accuracy: {accuracy:.2%}")

In [23]:
test(test_loader, model, criterion)

Average loss: 1.108807, accuracy: 81.73%
